In [1]:
from ultralytics import YOLO
import os
import random
import cv2
import numpy as np
import shutil
import supervision as sv
import cv2
import matplotlib.pyplot as plt




/home/mbergst/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [4]:
# Choose dataset 1 or 2:
dataset = 1  # 1 or 2

if dataset == 1:
    data_folder = "1_train-val_1min_aalesund_from_start/"
    yaml_file = "data_1.yaml"
else:
    data_folder = "2_train-val_1min_after_goal/"
    yaml_file = "data_2.yaml"

In [5]:
# Not running

# Generate shuffled txt and img files for training and validation, 80% training and 20% validation

from shuffler import generate_shuffled_txt_img_files_for_train_val

base_path = f"data_yolov8/{data_folder}/"  
text_file_path = f"/datasets/tdt4265/other/rbk/{data_folder}/gt/gt.txt"  
all_images_path = f"/datasets/tdt4265/other/rbk/{data_folder}/img1"
img_width = 1920  
img_height = 1080  

generate_shuffled_txt_img_files_for_train_val(text_file_path, img_width, img_height, base_path, all_images_path)

# Generate test txt and img files

from data_converting import generate_txt_img_files_for_test

base_path = "data_yolov8/3_test_1min_hamkam_from_start/"  
text_file_path = "/datasets/tdt4265/other/rbk/3_test_1min_hamkam_from_start/gt/gt.txt"  
all_images_path = "/datasets/tdt4265/other/rbk/3_test_1min_hamkam_from_start/img1/"

generate_txt_img_files_for_test(text_file_path, img_width, img_height, base_path, all_images_path)


KeyboardInterrupt: 

In [6]:
# Last inn sist trente modell

def find_latest_model_with_best(base_path):
    detect_path = os.path.join(base_path, 'runs', 'detect')
    training_sessions = [os.path.join(detect_path, d) for d in os.listdir(detect_path) if os.path.isdir(os.path.join(detect_path, d))]
    
    # Liste for å holde stiene til alle 'best.pt'-filer
    best_model_paths = []

    for session in training_sessions:
        best_model_path = os.path.join(session, 'weights', 'best.pt')
        if os.path.exists(best_model_path):
            best_model_paths.append((best_model_path, os.path.getmtime(best_model_path)))

    if not best_model_paths:
        print("Ingen 'best.pt' fil funnet i noen av treningsøktene.")
        return None
    
    # Sorter basert på modifiseringstid og returner den nyeste
    best_model_paths.sort(key=lambda x: x[1], reverse=True)
    return best_model_paths[0][0]


base_path = '/work/mbergst/TDT4265_Project' 
latest_model_path_with_best = find_latest_model_with_best(base_path)
if latest_model_path_with_best:
    print(f"Siste 'best.pt' modell funnet på: {latest_model_path_with_best}")



model = YOLO(latest_model_path_with_best)


Siste 'best.pt' modell funnet på: /work/mbergst/TDT4265_Project/runs/detect/train6/weights/best.pt


In [24]:
# Object Detection

# Tren modellen på datasettet
results = model.train(data=yaml_file, epochs=3, batch=7, imgsz=(1920, 1080), project='/work/mbergst/TDT4265_Project/runs/detect')



New https://pypi.org/project/ultralytics/8.1.46 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.42 🚀 Python-3.10.12 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24195MiB)
engine/trainer: task=detect, mode=train, model=/work/mbergst/TDT4265_Project/runs/detect/train5/weights/best.pt, data=data_1.yaml, epochs=3, time=None, patience=100, batch=7, imgsz=(1920, 1080), save=True, save_period=-1, cache=False, device=None, workers=8, project=/work/mbergst/TDT4265_Project/runs/detect, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, a

train: Scanning /work/mbergst/TDT4265_Project/data_yolov8/1_train-val_1min_aalesund_from_start/labels/train.cache... 1441 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1441/1441 [00:00<?, ?it/s]
val: Scanning /work/mbergst/TDT4265_Project/data_yolov8/1_train-val_1min_aalesund_from_start/labels/val.cache... 361 images, 0 backgrounds, 0 corrupt: 100%|██████████| 361/361 [00:00<?, ?it/s]


Plotting labels to /work/mbergst/TDT4265_Project/runs/detect/train6/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0004921875), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 1920 train, 1920 val
Using 8 dataloader workers
Logging results to /work/mbergst/TDT4265_Project/runs/detect/train6
Starting training for 3 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/3      11.1G     0.7397     0.5625     0.8382        275       1920: 100%|██████████| 206/206 [00:28<00:00,  7.23it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:01<00:00, 14.45it/s]

                   all        361       8361       0.93      0.759      0.808      0.581


...

        3/3      12.6G     0.6846      0.467      0.828        372       1920: 100%|██████████| 206/206 [00:27<00:00,  7.48it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:01<00:00, 17.53it/s]

                   all        361       8361      0.954      0.793      0.831      0.619



3 epochs completed in 0.025 hours.
Optimizer stripped from /work/mbergst/TDT4265_Project/runs/detect/train6/weights/last.pt, 6.5MB
Optimizer stripped from /work/mbergst/TDT4265_Project/runs/detect/train6/weights/best.pt, 6.5MB

Validating /work/mbergst/TDT4265_Project/runs/detect/train6/weights/best.pt...
Ultralytics YOLOv8.1.42 🚀 Python-3.10.12 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24195MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:04<00:00,  6.47it/s]


                   all        361       8361      0.951      0.793       0.83      0.619
                  ball        361        241       0.92      0.589      0.665      0.376
                player        361       8120      0.981      0.997      0.995      0.862
Speed: 0.4ms preprocess, 2.9ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to /work/mbergst/TDT4265_Project/runs/detect/train6


In [25]:
# Evaluér modellens prestasjon på valideringssettet
val_results = model.val()


Ultralytics YOLOv8.1.42 🚀 Python-3.10.12 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24195MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /work/mbergst/TDT4265_Project/data_yolov8/1_train-val_1min_aalesund_from_start/labels/val.cache... 361 images, 0 backgrounds, 0 corrupt: 100%|██████████| 361/361 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 52/52 [00:03<00:00, 16.08it/s]


                   all        361       8361      0.961      0.793      0.833      0.628
                  ball        361        241      0.941      0.589      0.672      0.389
                player        361       8120      0.981      0.997      0.995      0.867
Speed: 0.5ms preprocess, 2.9ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /work/mbergst/TDT4265_Project/runs/detect/train62


In [8]:
# Test modellen på testsettet
test_results = model.val(data='test.yaml')

Ultralytics YOLOv8.1.42 🚀 Python-3.10.12 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24195MiB)


val: Scanning /work/mbergst/TDT4265_Project/data_yolov8/3_test_1min_hamkam_from_start/labels/test... 1802 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1802/1802 [00:00<00:00, 4325.07it/s]

val: New cache created: /work/mbergst/TDT4265_Project/data_yolov8/3_test_1min_hamkam_from_start/labels/test.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 113/113 [00:14<00:00,  7.84it/s]


                   all       1802      42192      0.627      0.658      0.607      0.446
                  ball       1802       1412      0.333      0.356      0.242      0.129
                player       1802      40780       0.92      0.961      0.971      0.762
Speed: 0.6ms preprocess, 3.3ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /home/mbergst/Desktop/TDT4265/project/ultralytics/runs/detect/val2


In [14]:
# Object tracking
all_frames = f"/work/mbergst/TDT4265_Project/data_yolov8/{data_folder}/images/all_frames"
frame_paths = sorted([os.path.join(all_frames, f) for f in os.listdir(all_frames) if f.endswith('.jpg')])



In [15]:
# Iterere over hver frame og bruk track metoden
for frame_path in frame_paths:
    # model.track kan være en generator, så vi bruker en løkke for å hente resultatene
    for result in model.track(frame_path, persist=True, stream=True, line_width=1):
        annotated_frame = result.plot(font_size=1, line_width=1)
        cv2.imshow('frame', annotated_frame)
        cv2.waitKey(1)  # Venter en kort tid så bildet kan oppdateres
        # Venter en kort tid så bildet kan oppdateres
        # Hvis 'ESC' (ASCII 27) er trykket, avslutt løkken
        if cv2.waitKey(1) & 0xFF == 27:
            break

cv2.destroyAllWindows()  # Lukker alle OpenCV-vinduene


image 1/1 /work/mbergst/TDT4265_Project/data_yolov8/2_train-val_1min_after_goal/images/all_frames/000001.jpg: 384x640 11 persons, 61.8ms
Speed: 0.9ms preprocess, 61.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /work/mbergst/TDT4265_Project/data_yolov8/2_train-val_1min_after_goal/images/all_frames/000002.jpg: 384x640 11 persons, 3.5ms
Speed: 1.0ms preprocess, 3.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /work/mbergst/TDT4265_Project/data_yolov8/2_train-val_1min_after_goal/images/all_frames/000003.jpg: 384x640 13 persons, 3.5ms
Speed: 1.1ms preprocess, 3.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /work/mbergst/TDT4265_Project/data_yolov8/2_train-val_1min_after_goal/images/all_frames/000004.jpg: 384x640 13 persons, 3.5ms
Speed: 1.0ms preprocess, 3.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /work/mbergst/TDT4265_Project/data_yolov8/2_train-val_1

In [26]:
boxann = sv.BoxAnnotator(
    thickness=1,
    text_thickness=1,
    text_scale=0.5,
    
)

for frame_path in frame_paths:
    # model.track kan være en generator, så vi bruker en løkke for å hente resultatene
    for result in model.track(frame_path, persist=True, stream=True, line_thickness=1):
        frame = result.orig_img
        
        detections = sv.Detections.from_ultralytics(result)
        

        frame = boxann.annotate(scene=frame, detections=detections)
        cv2.imshow('yolov8', frame)
        cv2.waitKey(1)  # Venter en kort tid så bildet kan oppdateres

cv2.destroyAllWindows()  # Lukker alle OpenCV-vinduene

WARNING ⚠️ 'line_thickness' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'line_width' instead.

image 1/1 /work/mbergst/TDT4265_Project/data_yolov8/1_train-val_1min_aalesund_from_start/images/all_frames/000001.jpg: 1088x1920 22 players, 4.5ms


SupervisionWarnings: BoxAnnotator is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead


error: OpenCV(4.9.0) /io/opencv/modules/highgui/src/window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [ ]:
for frame in frame_paths:
    results = model.track(frame, persist=True)
    boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
    ids = results[0].boxes.id.cpu().numpy().astype(int)
    for box, id in zip(boxes, ids):
        cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
        cv2.putText(
            frame,
            f"Id {id}",
            (box[0], box[1]),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 0, 255),
            2,
        )
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break


image 1/1 /work/mbergst/project/data_yolov8/1_train-val_1min_aalesund_from_start/images/all_frames/000001.jpg: 1088x1920 2 players, 3.2ms
Speed: 21.6ms preprocess, 3.2ms inference, 0.6ms postprocess per image at shape (1, 3, 1088, 1920)


error: OpenCV(4.9.0) :-1: error: (-5:Bad argument) in function 'rectangle'
> Overload resolution failed:
>  - img is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'img'
>  - img is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'img'
